<a href="https://colab.research.google.com/github/Isaac-Jacobson/snow/blob/main/fallSpeed.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import cv2
from cv2 import threshold
import numpy as np
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt 
from skimage import measure
import math
import scipy
from google.colab.patches import cv2_imshow
from cv2 import HoughLines

In [ ]:
bg1 = cv2.imread('bg_1.bmp')
bg2 = cv2.imread('bg_2.bmp')
flake1 = cv2.imread('flake1_1.bmp')
flake2 = cv2.imread('flake1_2.bmp')

In [ ]:
cv2_imshow(bg1)
cv2_imshow(bg2)
cv2_imshow(flake1)
cv2_imshow(flake2)
#cv2.imshow(bg1)
#cv2.imshow(bg2)
#cv2.imshow(flake1)
#cv2.imshow(flake2)

In [ ]:
img1 = cv2.subtract(flake1, bg1)
img2 = cv2.subtract(flake2, bg2)

#Thresholds
ret1, binImg1 = threshold(img1, 133, 255, cv2.THRESH_BINARY)
ret2, binImg2 = threshold(img2, 33, 255, cv2.THRESH_BINARY)

#Thresholds
blur1 = cv2.GaussianBlur(binImg1,(19,19),0)
img1 = cv2.threshold(blur1, 100, 255, cv2.THRESH_BINARY)[1]
blur2 = cv2.GaussianBlur(binImg2,(19,19),0)
img2 = cv2.threshold(blur2, 100, 255, cv2.THRESH_BINARY)[1]

In [ ]:
cv2_imshow(img1)
cv2_imshow(img2)
#cv2.imshow(img1)
#cv2.imshow(img2)

In [ ]:
props = measure.regionprops(img1)
areas = [0]*len(props)
centroids = [0]*len(props)
for i in range(0, len(props)):
  areas[i] = props[i].area
  centroids[i] = props[i].centroid[1:2]
radius = math.sqrt(sum(areas) / math.pi)

In [ ]:
edges = img1 - scipy.ndimage.morphology.binary_dilation(img1)
xsum = 0
ysum = 0
count = 0
for x in range(1, 1200):
    for y in range(1, 1920):
        if(img1[x][y].any() == 1):
            xsum = xsum+x;
            ysum = ysum+y;
            count = count+1;          
y1 = round(xsum/count);
x1 = round(ysum/count);


In [ ]:
plt.imshow(edges); 
plt.plot(x1,y1,'rx');

In [ ]:
edges2 = img1 - scipy.ndimage.morphology.binary_dilation(img1)

In [ ]:
max_len = 0
sum1x = 0
sum1y = 0
sum2x = 0
sum2y = 0

In [ ]:
#Thresholds
edgesCanny = cv2.Canny(img2, 10, 200, 3)
lines = cv2.HoughLines(edgesCanny, 1, np.pi/180, 100, None, 0, 0)
for rho,theta in lines[0]:
    a = np.cos(theta)
    b = np.sin(theta)
    x0 = a*rho
    y0 = b*rho
    x1 = int(x0 + 1000*(-b))
    y1 = int(y0 + 1000*(a))
    x2 = int(x0 - 1000*(-b))
    y2 = int(y0 - 1000*(a))
    sum1x += x1
    sum1y += y1
    sum2x += x2
    sum2y += y2
    cv2.line(img2,(x1,y1),(x2,y2),(0,0,255),2)
cv2_imshow(img2)
#cv2.imshow(img2)

In [ ]:
avg1x = round(sum1x/len(lines));
avg1y = round(sum1y/len(lines));
avg2x =  round(sum2x/len(lines));
avg2y =  round(sum2y/len(lines));

In [ ]:
d2 = avg2y - avg1y;
dx2 = avg2x - avg1x;
angle2 = math.atan(d2/dx2);

In [ ]:
ry = radius*math.sin(angle2);
rx = radius*math.cos(angle2);

avg1x = avg1x+rx;
avg1y = avg1y+ry;

avg2x = avg2x-rx;
avg2y = avg2y-ry;

In [ ]:
zz = [(avg1x, avg1y),(avg2x, avg2y)];
d1 = avg1y - y1
dx1 = avg1x - x1;

In [ ]:
angle1 = math.atan(d1/dx1)
d2 = avg2y - avg1y
dx2 = avg2x - avg1x
speed1 = d1/0.027
speed2 = d2/0.021

print("Speed1 : " + str(abs(speed1)))
#print("Speed2 : " + str(abs(speed2)))

Speed1 : 32408.01344882682
